# Kannada to English Machine Translation

Importing/Downloading Libraries

In [1]:
#in case conda environment not available locally, run these commands, or open this in colab
%pip install tensorflow
%pip install nltk

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk.translate.bleu_score as bleu
import random
import string
from sklearn.model_selection import train_test_split
import os
import time

## Creating CSV Files

In [10]:
small_kn = open('./data/small.kn', encoding='utf-8').readlines()
small_en = open('./data/small.en').readlines()
for pair in list(zip(small_en[:5], small_kn[:5])):
    print(pair[0][:-2],'\t',pair[1][:-2])

Such videos are aplenty on social media channels 	 ಸಾಮಾಜಿಕ ಜಾಲತಾಣಗಳಲ್ಲಿ ಅದೆಷ್ಟೊ ವಿಡಿಯೋಗಳು ಹರಿದಾಡುತ್ತಿರುತ್ತವೆ
This incident happened within the limits of Mudhol Police Station 	 ಮುಳಗುಂದ ಪೊಲೀಸ್ ಠಾಣಾ ವ್ಯಾಪ್ತಿಯಲ್ಲಿ ಈ ದುರ್ಘಟನೆ ನಡೆದಿದೆ
The situation is currently under control, he said 	 ಪ್ರಸ್ತುತ ಪರಿಸ್ಥಿತಿ ನಿಯಂತ್ರಣದಲ್ಲಿದೆ ಎಂದು ತಿಳಿಸಿದ್ದಾರೆ
Choose a Bookmark to be your Home Page. If you choose a folder, the Bookmarks in that folder will be opened in Tabs 	 ನಿಮ್ಮ ನೆಲೆ ಪುಟವಾಗಿಸಲು ಒಂದು ಬುಕ್‌ಮಾರ್ಕ್ ಅನ್ನು ಆರಿಸಿ. ನೀವು ಒಂದು ಕೋಶವನ್ನು ಆರಿಸಿದರೆ, ಅದರಲ್ಲಿನ ಬುಕ್‌ಮಾರ್ಕುಗಳು ಪ್ರತ್ಯೇಕ ಹಾಳೆಗಳಾಗಿ ತೆರೆಯಲ್ಪಡುತ್ತವೆ
Several social organisations are also being involved 	 ಇದಕ್ಕೆ ಸಾಮಾಜಿಕವಾದ ಹಲವು ವ್ಯವಸ್ಥೆಗಳು ಪೂರಕವಾಗಿ ಕಾರ್ಯ ನಿರ್ವಹಿಸುತ್ತವೆ


In [30]:
with open('./data/kan_eng.csv','w') as f:
    for pair in list(zip(small_en, small_kn)):
        new_pair0 = pair[0].replace(',','')
        new_pair1 = pair[1].replace(',', '')
        new_pair0 = new_pair0.replace('\'', '')
        new_pair1 = new_pair1.replace('\'', '')
        new_pair0 = new_pair0.replace('\n','')
        new_pair1 = new_pair1.replace('\n','')
        str = new_pair0 + ',' + new_pair1
        f.write(str)
        f.write('\n') 

In [31]:
df = pd.read_csv('./data/kan_eng.csv',names=['English','Kannada'])

In [32]:
df.head()

,English,Kannada
0,Such videos are aplenty on social media channels.,ಸಾಮಾಜಿಕ ಜಾಲತಾಣಗಳಲ್ಲಿ ಅದೆಷ್ಟೊ ವಿಡಿಯೋಗಳು ಹರಿದಾಡು...
1,This incident happened within the limits of Mu...,ಮುಳಗುಂದ ಪೊಲೀಸ್ ಠಾಣಾ ವ್ಯಾಪ್ತಿಯಲ್ಲಿ ಈ ದುರ್ಘಟನೆ ನ...
2,The situation is currently under control he said.,ಪ್ರಸ್ತುತ ಪರಿಸ್ಥಿತಿ ನಿಯಂತ್ರಣದಲ್ಲಿದೆ ಎಂದು ತಿಳಿಸಿ...
3,Choose a Bookmark to be your Home Page. If you...,ನಿಮ್ಮ ನೆಲೆ ಪುಟವಾಗಿಸಲು ಒಂದು ಬುಕ್‌ಮಾರ್ಕ್ ಅನ್ನು ಆ...
4,Several social organisations are also being in...,ಇದಕ್ಕೆ ಸಾಮಾಜಿಕವಾದ ಹಲವು ವ್ಯವಸ್ಥೆಗಳು ಪೂರಕವಾಗಿ ಕಾ...
